# Solution for Modifiability

_This challenge was inspired by TSJ CTF 2022's [Cipher Switching Service](https://github.com/maple3142/My-CTF-Challenges/tree/master/TSJ%20CTF%202022/Cipher%20Switching%20Service) where you had to swap between ElGamal and RSA, and I thought we could do a similar thing with AES modes._

The trick here is that [the CFB segment size defaults to 8](https://pycryptodome.readthedocs.io/en/latest/src/cipher/classic.html#cfb-mode), so that if you CFB-decrypt a large array of `00`s, then everything after the first 16 bytes will just be a single byte repeated indefinitely. This repeated byte is in fact the first byte when you encrypt 16 `00`s into the block cipher. The value of this byte is unknown, so let's call it `g`.

We use this in conjunction with OFB, which is basically a xorstream whose (1-indexed) `i`th block is simply the IV encrypted `i` times. Now, if we converted that array of `00`s from CFB to OFB, this would effectively give all values of the IV encrypted 2 or more times, XORed with this unknown byte `g`. But this is just a factor of 256 which we can easily brute force.

The only ingredient left is the single-encrypted IV, which means we need to send back the doubly-encrypted IV through CBC to decrypt it once. Thus, the plan of attack is as follows:
1. Convert a long stream of `00`s from CFB to OFB (gives us a xorstream from the second block onwards, also XORed with `g`)
2. For each value of `g`, get CBC to decrypt the triply-encrypted IV as well as the doubly-encrypted IV. The former tells us if we have the correct value of `g` (since we know the double-encrypted IV), while the latter will tell us the singly-encrypted IV. At the end of this we know `g` as well the entire OFB xorstream.
3. Since we have a xorstream, we effectively know how to encrypt and decrypt OFB, so send the message in OFB to get the flag back in OFB.

In [1]:
from pwn import *
with remote('fun.chall.seetf.sg', 30003) as sh:
    def get(modes, ct):
        sh.recvuntil(b'use: ')
        sh.sendline(f'{modes} {ct.hex()}'.encode())
        return bytes.fromhex(sh.recvline().decode())

    tmp = get('CFB OFB', b'\0'*8208)
    z = b'\0'*16 + b''.join(xor(tmp[16:48],i) for i in range(256))

    foo = get('CBC OFB', z)
    foo = xor(foo[16:], tmp[16:], z[:-16])

    g = next(i for i in range(256) if foo[i*32+16:i*32+32] == tmp[16:32])
    ofb_stream = xor(foo[g*32:g*32+16] + tmp[16:], g)

    ofb = lambda x:xor(x,ofb_stream,cut='min')
    print(ofb(get('OFB OFB', ofb(b'I Can Has Flag Plz?'))))

[x] Opening connection to fun.chall.seetf.sg on port 30003
[x] Opening connection to fun.chall.seetf.sg on port 30003: Trying 34.131.197.225
[+] Opening connection to fun.chall.seetf.sg on port 30003: Done
b'SEE{MODErn_problems_require_MODErn_solutions_8aff35d760e2c8ed}'
[*] Closed connection to fun.chall.seetf.sg port 30003
